<a href="https://colab.research.google.com/github/Cairo-Henrique/BQC-Quantum-Tech/blob/main/Bloco_1/BQC_Aula_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brazil Quantum Camp

**Aula 3**: Introdução à Programação I

In [ ]:
import numpy as np
import plotly.express as px
from functools import reduce

In [ ]:
def get_op_n_qubits(gate_matrix, target_qubit, total_qubits):
    """Constrói a matriz de operação para N qubits"""

    # 1. Cria lista de Identidades: [I, I, I, ...]
    gates = [np.eye(2, dtype=complex)] * total_qubits

    # 2. Substitui a Identidade no
    # alvo pela porta desejada
    gates[target_qubit] = gate_matrix

    # 3. Faz o kron de tudo: I kron I kron U kron I ...
    # reduce aplica kron acumulativamente na lista
    full_op = reduce(np.kron, gates)

    return full_op

def get_cnot_matrix(n_qubits, control, target):
    """Constrói a matriz CNOT para N qubits item a item."""

    size = 2**n_qubits
    control = n_qubits - 1 - control  # Ajusta índice
    target = n_qubits - 1 - target  # Ajusta índice
    U = np.zeros((size, size), dtype=complex)
    # Itera sobre todos os estados da base (colunas)
    for col in range(size):
        # Verifica se o bit de controle está ativo
        if (col >> control) & 1:
            # Se sim, o destino (linha) é o índice com o bit alvo invertido
            row = col ^ (1 << target)
        else:
            # Se não, o estado não muda
            row = col
        # Define a transição |row><col|
        U[row, col] = 1.0
    return U

def plot(state: np.ndarray, **kwargs):
    """Plota o estado quântico usando Plotly Express."""

    # 1. Preparando os dados
    # flatten(): Transforma matriz (4,1) em array plano (4,)
    raio = np.abs(state.flatten())
    angulo = np.angle(state.flatten())
    # 2. Gerando labels (|00>, |01>...) dinamicamente
    n_qubits = int(np.log2(len(state)))
    estados = [f"|{i:0{n_qubits}b}>" for i in range(len(raio))]
    # 3. Plotando
    # x=Estado, y=Amplitude, color=Fase
    fig = px.bar(x=estados, y=raio, color=angulo, **kwargs)
    return fig

In [ ]:
# Operadores unitários
I = np.array([[1, 0],
              [0, 1]])
X = np.array([[0, 1],
              [1, 0]])
Y = np.array([[0, -1j],
              [1j, 0]])
Z = np.array([[1, 0],
              [0, -1]])
H = np.array([[1,  1],
              [1, -1]])  / np.sqrt(2)
CNOT = np.kron(I, X)

# Vetores de estado
ket_0 = np.array([[1], [0]], dtype=complex)
ket_1 = np.array([[0], [1]], dtype=complex)
ket_plus = np.array([[1], [1]], dtype=complex) / np.sqrt(2)
ket_minus = np.array([[1],[-1]], dtype=complex) / np.sqrt(2)

In [ ]:
# Exemplo com CNOT nos qubits 0 e 1
ket_psi = CNOT@(np.kron(ket_0,ket_1))
print(np.kron(H,H) @ np.kron(ket_0, ket_1))

[[ 0.5+0.j]
 [-0.5+0.j]
 [ 0.5+0.j]
 [-0.5+0.j]]


In [ ]:
plot(np.kron(H,H) @ np.kron(ket_0, ket_1))

## 🛠️ Mão na Massa: O Core do Simulador

Vamos implementar a **API** (Interface) do nosso simulador.

1.  **`create_state(n_qubits: int) -> np.ndarray`**
    Inicializa o vetor de estado quântico com dimensão $2^N$.
    *Estado inicial:* $\ket{0}^{\otimes N} = \ket{00\dots0}$.

2.  **`apply_gate(state: np.ndarray, gate_name: str, target: int) -> np.ndarray`**
    Aplica uma porta de 1 qubit em um alvo específico.
    *Matemática:* Constrói a matriz usando $I \otimes \dots \otimes U \otimes \dots \otimes I$.

3.  **`apply_cnot(state: np.ndarray, control: int, target: int) -> np.ndarray`**
    Aplica a porta controlada (CNOT) entre dois qubits arbitrários.
    Permite criar emaranhamento entre os qubits.

In [ ]:
def create_state(n_qubits: int) -> np.ndarray:
    """Cria um estado quântico inicializado no estado
    |0...0> para um dado número de qubits."""
    qubits = n_qubits * [ket_0]
    estado_composto = reduce(np.kron, qubits)
    return estado_composto


def apply_gate(state: np.ndarray, gate_name: str, target: int) -> np.ndarray:
    """Aplica uma porta quântica de um qubit a um estado quântico."""
    n_qubits = int(np.log2(len(state)))
    operator = get_op_n_qubits(eval(gate_name), target, n_qubits)
    return operator @ state


def apply_cnot(state: np.ndarray, control: int, target: int) -> np.ndarray:
    """Aplica a porta CNOT a um estado quântico."""
    n_qubits = int(np.log2(len(state)))
    operator = get_cnot_matrix(n_qubits, control, target)
    return operator @ state

## 🧪 **Exercício**: A Base de Bell Completa

Os estados de Bell ($\left|\Phi^\pm\right>, \left|\Psi^\pm\right>$) formam uma base
ortonormal de estados maximamente emaranhados.

**O Desafio**:
Use suas funções `create_state`, `apply_gate` e `apply_cnot`
para gerar e visualizar os 4 estados.


$$
\begin{aligned}
    \left|\Phi^+\right> = \frac{\left|00\right>+\left|11\right>}{\sqrt{2}} \qquad \qquad \left|\Phi^-\right> = \frac{\left|00\right>-\left|11\right>}{\sqrt{2}}\\
    \left|\Psi^+\right> = \frac{\left|01\right>+\left|10\right>}{\sqrt{2}} \qquad \qquad \left|\Psi^-\right> = \frac{\left|01\right>-\left|10\right>}{\sqrt{2}}
\end{aligned}
$$

In [ ]:
initial_state = create_state(2)

# |phi+>
ket_phi_plus = apply_gate(initial_state, 'H', 0)
ket_phi_plus = apply_cnot(ket_phi_plus, 0, 1)
print(ket_phi_plus)

# |phi->
ket_phi_minus = apply_gate(initial_state, 'X', 0)
ket_phi_minus = apply_gate(ket_phi_minus, 'H', 0)
ket_phi_minus = apply_cnot(ket_phi_minus, 0, 1)
print(ket_phi_minus)

# |psi+>
ket_psi_plus = apply_gate(initial_state, 'X', 1)
ket_psi_plus = apply_gate(ket_psi_plus, 'H', 0)
ket_psi_plus = apply_cnot(ket_psi_plus, 0, 1)
print(ket_psi_plus)

# |psi->
ket_psi_minus = apply_gate(initial_state, 'X', 0)
ket_psi_minus = apply_gate(ket_psi_minus, 'X', 1)
ket_psi_minus = apply_gate(ket_psi_minus, 'H', 0)
ket_psi_minus = apply_cnot(ket_psi_minus, 0, 1)
print(ket_psi_minus)

[[0.70710678+0.j]
 [0.        +0.j]
 [0.        +0.j]
 [0.70710678+0.j]]
[[ 0.70710678+0.j]
 [ 0.        +0.j]
 [ 0.        +0.j]
 [-0.70710678+0.j]]
[[0.        +0.j]
 [0.70710678+0.j]
 [0.70710678+0.j]
 [0.        +0.j]]
[[ 0.        +0.j]
 [ 0.70710678+0.j]
 [-0.70710678+0.j]
 [ 0.        +0.j]]


In [ ]:
plot(ket_phi_plus)

In [ ]:
plot(ket_phi_minus)

In [ ]:
plot(ket_psi_plus)

In [ ]:
plot(ket_psi_minus)